In [ ]:
import pandas as pd
import fastf1
from fastf1 import plotting
import matplotlib.pyplot as plt
import seaborn as sns
from src.plotset import setup_plot
from src.utils import fuel_correction

setup_plot()

In [ ]:
fastf1.Cache.enable_cache('./f1_cache')
fastf1.Cache.get_cache_info()

In [ ]:
race = fastf1.get_session(2025,'Spain','R')
race.load()

In [ ]:
ver_df = race.laps.pick_drivers('VER')

In [ ]:
df = pd.DataFrame(ver_df[['LapTime','LapNumber','Stint','Compound','TyreLife','TrackStatus','IsAccurate']])

In [ ]:
df['LapTime'] = df['LapTime'].dt.total_seconds()

In [ ]:
df_filtered = df[df.IsAccurate].reset_index(drop=True)

In [ ]:
df_filtered['Compound_Stint'] = df_filtered.apply(lambda x: x['Compound'] + '_Stint' + str(x['Stint']),axis=1)

In [ ]:
df_filtered.Compound_Stint.unique()

In [ ]:
plotting.get_compound_mapping(race)

In [ ]:
new_pallete = {'SOFT_Stint1.0': '#da291c',
 'SOFT_Stint2.0': '#da291c',
 'SOFT_Stint4.0': '#da291c',
 'MEDIUM_Stint3.0': '#ffd12e',
 'HARD_Stint5.0': '#f0f0ec'}

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))

sns.lineplot(data=df_filtered,x='LapNumber',y='LapTime',hue='Compound_Stint',
             palette=new_pallete,marker='o',markeredgecolor=None,legend=False,ax=ax)

In [ ]:
df_filtered['FC_LapTime'] = fuel_correction(race,df_filtered)

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))

sns.lineplot(data=df_filtered,x='LapNumber',y='FC_LapTime',hue='Compound_Stint',
             palette=new_pallete,marker='o',markeredgecolor=None,legend=False,ax=ax)

In [ ]:
df_stint = df_filtered.drop(['Compound_Stint','TrackStatus'],axis=1).groupby(['Stint','Compound','TyreLife']).mean()

In [ ]:
import statsmodels.api as sm

In [ ]:
df_reg = df_stint.xs((3,'MEDIUM'))
df_reg

In [ ]:
X = df_reg.index
X = sm.add_constant(X)
y = df_reg['FC_LapTime']

In [ ]:
model = sm.OLS(y, X).fit()

In [ ]:
print(model.summary())